# LoC/RoC detection for EEG signal under general anesthesia (Propofol/Sevoflurane)

**Goal**

- Load the data
- Compute the LoC and RoC 
- Visualize LoC and RoC on a spectrogram



In [ ]:
import mne
import numpy as np
from boost_loc_roc.LoC_RoC import *

In [ ]:
# Here we load one EEG recording
filename = 'data/raw/example_eeg.fif'

# load the raw filegit commit 
file_tmp = mne.io.read_raw_fif(filename)

# LoC/RoC detection 


**Remark**
The following code is used on ICEO file but should be easily extended to other dataset. 

This step can be embedded directly in the loop dealing with other segmentation processes.

The following cell is thus only for pedagogical purposes, but can also serve as an inspection step. 

In [ ]:
%%time

# Truncate the file
file_tmp = Truncate_fif(file_tmp)

# Calculate the duration of the file in seconds
duration_sec = file_tmp.n_times / file_tmp.info['sfreq']
print('sfreq', file_tmp.info['sfreq'])

# Check if the duration is greater than 5 minutes
if duration_sec > 5 * 60:
    # Resample the file to 63 Hz
    file_tmp = file_tmp.resample(63)
    
    # Extract sampling frequency and create a time vector
    Fs = file_tmp.info['sfreq']  # Hz
    time = np.linspace(0, file_tmp.get_data().shape[1]/Fs, file_tmp.get_data().shape[1], endpoint=False)
    
    # Extract the EEG data and convert to microvolts
    Fp2 = file_tmp.get_data()[1,:]*10**6  # 10*6 because .fif is saved in Volts
    
    # Compute LoC and RoC
    time_loc, time_roc, t_proba, proba, tmp1, tmp2 = extract_loc_roc(file_tmp)

    # Visualize Spectrogram, EEG, and proba
    plot_spectrogram_proba(time_loc, time_roc, Fp2, Fs, time, t_proba, proba)

## Check sigmoids fit on probabilities

In [ ]:
%%time

# Truncate the file
file_tmp = Truncate_fif(file_tmp)

# Calculate the duration in seconds
duration_sec = file_tmp.n_times / file_tmp.info['sfreq']
print('sfreq', file_tmp.info['sfreq'])

# Check if the duration is greater than 5 minutes
if duration_sec > 5 * 60:
    # Resample the file to 63 Hz
    file_tmp = file_tmp.resample(63)
    
    # Extract the sampling frequency and create a time vector
    Fs = file_tmp.info['sfreq']  # Hz
    time = np.linspace(0, file_tmp.get_data().shape[1]/Fs, file_tmp.get_data().shape[1], endpoint=False)
    Fp2 = file_tmp.get_data()[1,:]*10**6  # 10*6 because .fif is saved in Volts

    # Extract parameters and data for LoC and RoC
    time_loc, time_roc, t_proba, proba, LoC_params, RoC_params = extract_loc_roc(file_tmp)
 
    # Visualize Spectrogram, EEG, and proba
    plot_spectrogram_debug(time_loc, time_roc, Fp2, Fs, time, t_proba, proba, LoC_params, RoC_params, filename)

## Applying Volt/microVolt unit correction 

==> use check_and_rescale_units function.

In [ ]:
# Import the check_and_rescale_units function from the boost_loc_roc.miscelaneous module
from boost_loc_roc.miscelaneous import check_and_rescale_units

# Define the filename variable with the path to the EEG data file
filename = 'data/raw/example_eeg.fif'

# Call the check_and_rescale_units function with the specified filename and threshold value of 5 uV
check_and_rescale_units(filename, threshold=5)